In [1]:
import sys

sys.path.append("/home/blair/projects/yabte_cpp/build/Debug/pybind")

import pandas as pd
import pyarrow as pa
import yabte
import yabte.backtest as ybt
import yabte_cpp_backtest as ybt_cpp
from yabte.tests._helpers import generate_nasdaq_dataset
from yabte.utilities.strategy_helpers import crossover
from yabte.utilities.plot.plotly.strategy_runner import plot_strategy_runner_result

ybt_cpp.__version__, yabte.__version__

('"Tue Apr  9 15:01:03 2024"', '0.4.0')

In [2]:
class SMAXO(ybt_cpp.Strategy):
    def __init__(self):
        ybt_cpp.Strategy.__init__(self)

    def clone(self):
        cloned = SMAXO.__new__(SMAXO)
        ybt_cpp.Strategy.__init__(cloned, self)
        cloned.__dict__.update(self.__dict__)
        return cloned

    def extend_data(self, data):
        # enhance data with simple moving averages

        df = data.to_pandas()

        p = self.params
        days_short = p["days_short"]
        days_long = p["days_long"]

        close_sma_short = (
            df.loc[:, (slice(None), "Close")]
            .rolling(days_short)
            .mean()
            .rename({"Close": "CloseSMAShort"}, axis=1, level=1)
        )
        close_sma_long = (
            df.loc[:, (slice(None), "Close")]
            .rolling(days_long)
            .mean()
            .rename({"Close": "CloseSMALong"}, axis=1, level=1)
        )
        new_df = pd.concat([close_sma_short, close_sma_long], axis=1).sort_index(axis=1)

        return pa.Table.from_pandas(new_df)

    def init(self): ...

    def on_close(self):
        # create some orders

        p = self.params
        days_long = p["days_long"]

        if (L := len(self.data)) < days_long + 1:
            return

        # for symbol in ["GOOG", "MSFT"]:
        for symbol in ["GOOG"]:
            key_long = f"('{symbol}', 'CloseSMALong')"
            key_short = f"('{symbol}', 'CloseSMAShort')"

            close_sma_short = self.data[key_short].slice(L - 2, 2).to_pandas()
            close_sma_long = self.data[key_long].slice(L - 2, 2).to_pandas()

            if crossover(close_sma_short, close_sma_long):
                self.orders.append(ybt_cpp.SimpleOrder(asset_name=symbol, size=-100))
            elif crossover(close_sma_long, close_sma_short):
                self.orders.append(ybt_cpp.SimpleOrder(asset_name=symbol, size=100))
                
# load some data
assets_py, df_combined = generate_nasdaq_dataset()

# convert to arrow table
table_ar = pa.Table.from_pandas(df_combined)

# recreate cpp assets from python assets
assets = [ybt_cpp.OHLCAsset(name=a.name) for a in assets_py]

# create a strategy
strategies = [SMAXO()]

# create a book with 100000 cash
books = [ybt_cpp.Book(name="Main", cash=100000)]

# run the strategy
sr1 = ybt_cpp.StrategyRunner(table_ar, assets, strategies, books)
srr1 = sr1.run(dict(days_short=10, days_long=20))                

I20240410 21:46:11.492758 893111 StrategyRunner.cpp:40] Running strategy runner
I20240410 21:46:11.503059 893111 StrategyRunner.cpp:77] Extending data for strategy
I20240410 21:46:11.514972 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.540762 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.576054 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.610029 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.631928 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.644712 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.712435 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.729684 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.777037 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.835713 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.850883 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.926810 893111 Order.cpp:69] SimpleOrder::apply()
I20240410 21:46:11.9

In [17]:
srr1.book_history.rename_columns(["Date", "('Main', 'cash')", "('Main', 'mtm')", "('Main', 'total')"])

pyarrow.Table
Date: timestamp[ns] not null
('Main', 'cash'): double not null
('Main', 'mtm'): double not null
('Main', 'total'): double not null
----
Date: [[2018-01-02 00:00:00.000000000,2018-01-03 00:00:00.000000000,2018-01-04 00:00:00.000000000,2018-01-05 00:00:00.000000000,2018-01-08 00:00:00.000000000,...,2022-12-23 00:00:00.000000000,2022-12-27 00:00:00.000000000,2022-12-28 00:00:00.000000000,2022-12-29 00:00:00.000000000,2022-12-30 00:00:00.000000000]]
('Main', 'cash'): [[100000,100000,100000,100000,100000,...,95169,95169,95169,95169,95169]]
('Main', 'mtm'): [[0,0,0,0,0,...,8981,8793,8646,8895,8873]]
('Main', 'total'): [[100000,100000,100000,100000,100000,...,104150,103962,103815,104064,104042]]

In [3]:
srr1.book_history.to_pandas()

,"('Main', 'ts')","('Main', 'cash')","('Main', 'mtm')","('Main', 'total')"
0,2018-01-02,100000.0,0.0,100000.0
1,2018-01-03,100000.0,0.0,100000.0
2,2018-01-04,100000.0,0.0,100000.0
3,2018-01-05,100000.0,0.0,100000.0
4,2018-01-08,100000.0,0.0,100000.0
...,...,...,...,...
1254,2022-12-23,95169.0,8981.0,104150.0
1255,2022-12-27,95169.0,8793.0,103962.0
1256,2022-12-28,95169.0,8646.0,103815.0
1257,2022-12-29,95169.0,8895.0,104064.0


In [4]:
srr1.book_history#.to_pandas().loc[:, "('Main', 'mtm')"].plot()

pyarrow.Table
('Main', 'ts'): timestamp[ns] not null
('Main', 'cash'): double not null
('Main', 'mtm'): double not null
('Main', 'total'): double not null
----
('Main', 'ts'): [[2018-01-02 00:00:00.000000000,2018-01-03 00:00:00.000000000,2018-01-04 00:00:00.000000000,2018-01-05 00:00:00.000000000,2018-01-08 00:00:00.000000000,...,2022-12-23 00:00:00.000000000,2022-12-27 00:00:00.000000000,2022-12-28 00:00:00.000000000,2022-12-29 00:00:00.000000000,2022-12-30 00:00:00.000000000]]
('Main', 'cash'): [[100000,100000,100000,100000,100000,...,95169,95169,95169,95169,95169]]
('Main', 'mtm'): [[0,0,0,0,0,...,8981,8793,8646,8895,8873]]
('Main', 'total'): [[100000,100000,100000,100000,100000,...,104150,103962,103815,104064,104042]]

In [5]:
srr1.books[0].transactions

[<yabte_backtest_cpp.Trade ts=2018-02-08 00:00:00.000000000, total=-5148, desc=buy GOOG, quantity=100, price=51.48, asset_name=GOOG, >,
 <yabte_backtest_cpp.Trade ts=2018-02-28 00:00:00.000000000, total=5577, desc=sell GOOG, quantity=-100, price=55.77, asset_name=GOOG, >,
 <yabte_backtest_cpp.Trade ts=2018-03-26 00:00:00.000000000, total=-5160, desc=buy GOOG, quantity=100, price=51.6, asset_name=GOOG, >,
 <yabte_backtest_cpp.Trade ts=2018-04-19 00:00:00.000000000, total=5406, desc=sell GOOG, quantity=-100, price=54.06, asset_name=GOOG, >,
 <yabte_backtest_cpp.Trade ts=2018-05-04 00:00:00.000000000, total=-5164, desc=buy GOOG, quantity=100, price=51.64, asset_name=GOOG, >,
 <yabte_backtest_cpp.Trade ts=2018-05-14 00:00:00.000000000, total=5525, desc=sell GOOG, quantity=-100, price=55.25, asset_name=GOOG, >,
 <yabte_backtest_cpp.Trade ts=2018-07-03 00:00:00.000000000, total=-5590, desc=buy GOOG, quantity=100, price=55.9, asset_name=GOOG, >,
 <yabte_backtest_cpp.Trade ts=2018-07-16 00:00:

In [6]:
srr1.books[0].history

pyarrow.Table
ts: timestamp[ns] not null
cash: double not null
mtm: double not null
total: double not null
----
ts: [[2018-01-02 00:00:00.000000000,2018-01-03 00:00:00.000000000,2018-01-04 00:00:00.000000000,2018-01-05 00:00:00.000000000,2018-01-08 00:00:00.000000000,...,2022-12-23 00:00:00.000000000,2022-12-27 00:00:00.000000000,2022-12-28 00:00:00.000000000,2022-12-29 00:00:00.000000000,2022-12-30 00:00:00.000000000]]
cash: [[100000,100000,100000,100000,100000,...,95169,95169,95169,95169,95169]]
mtm: [[0,0,0,0,0,...,8981,8793,8646,8895,8873]]
total: [[100000,100000,100000,100000,100000,...,104150,103962,103815,104064,104042]]

In [7]:
class SMAXO(ybt.Strategy):
    def init(self):
        # enhance data with simple moving averages

        p = self.params
        days_short = p.get("days_short", 10)
        days_long = p.get("days_long", 20)

        close_sma_short = (
            self.data.loc[:, (slice(None), "Close")]
            .rolling(days_short)
            .mean()
            .rename({"Close": "CloseSMAShort"}, axis=1, level=1)
        )
        close_sma_long = (
            self.data.loc[:, (slice(None), "Close")]
            .rolling(days_long)
            .mean()
            .rename({"Close": "CloseSMALong"}, axis=1, level=1)
        )
        self.data = pd.concat(
            [self.data, close_sma_short, close_sma_long], axis=1
        ).sort_index(axis=1)

    def on_close(self):
        # create some orders

        # for symbol in ["GOOG", "MSFT"]:
        for symbol in ["GOOG"]:
            df = self.data[symbol]
            ix_2d = df.index[-2:]
            data = df.loc[ix_2d, ("CloseSMAShort", "CloseSMALong")].dropna()
            if len(data) == 2:
                if crossover(data.CloseSMAShort, data.CloseSMALong):
                    self.orders.append(ybt.SimpleOrder(asset_name=symbol, size=-100))
                elif crossover(data.CloseSMALong, data.CloseSMAShort):
                    self.orders.append(ybt.SimpleOrder(asset_name=symbol, size=100))


# load some data
assets, df_combined = generate_nasdaq_dataset()

# create a book with 100000 cash
book = ybt.Book(name="Main", cash="100000")

# run our strategy
sr2 = ybt.StrategyRunner(
    data=df_combined,
    assets=assets,
    strategies=[SMAXO()],
    books=[book],
)
srr2 = sr2.run(dict(days_short=10, days_long=20))

# see the trades or book history
th = srr2.transaction_history
bch = srr2.book_history.loc[:, (slice(None), "cash")]

# plot the trades against book value
#plot_strategy_runner_result(srr2, sr2)

In [8]:
srr2.books[0].transactions

[Trade(ts=Timestamp('2018-02-08 00:00:00'), total=Decimal('-5148.0000'), desc='buy GOOG', quantity=Decimal('100.00'), price=Decimal('51.48'), asset_name='GOOG', order_label=None),
 Trade(ts=Timestamp('2018-02-28 00:00:00'), total=Decimal('5577.0000'), desc='sell GOOG', quantity=Decimal('-100.00'), price=Decimal('55.77'), asset_name='GOOG', order_label=None),
 Trade(ts=Timestamp('2018-03-26 00:00:00'), total=Decimal('-5160.0000'), desc='buy GOOG', quantity=Decimal('100.00'), price=Decimal('51.60'), asset_name='GOOG', order_label=None),
 Trade(ts=Timestamp('2018-04-19 00:00:00'), total=Decimal('5406.0000'), desc='sell GOOG', quantity=Decimal('-100.00'), price=Decimal('54.06'), asset_name='GOOG', order_label=None),
 Trade(ts=Timestamp('2018-05-04 00:00:00'), total=Decimal('-5164.0000'), desc='buy GOOG', quantity=Decimal('100.00'), price=Decimal('51.64'), asset_name='GOOG', order_label=None),
 Trade(ts=Timestamp('2018-05-14 00:00:00'), total=Decimal('5525.0000'), desc='sell GOOG', quantity

In [9]:
srr2.book_history#.loc[:, (slice(None), "mtm")].plot()

Main                  
                cash     mtm     total
ts                                    
2018-01-02  100000.0     0.0  100000.0
2018-01-03  100000.0     0.0  100000.0
2018-01-04  100000.0     0.0  100000.0
2018-01-05  100000.0     0.0  100000.0
2018-01-08  100000.0     0.0  100000.0
...              ...     ...       ...
2022-12-23   95169.0  8981.0  104150.0
2022-12-27   95169.0  8793.0  103962.0
2022-12-28   95169.0  8646.0  103815.0
2022-12-29   95169.0  8895.0  104064.0
2022-12-30   95169.0  8873.0  104042.0

[1259 rows x 3 columns]

In [10]:
bch

,Main
,cash
ts,
2018-01-02,100000.0
2018-01-03,100000.0
2018-01-04,100000.0
2018-01-05,100000.0
2018-01-08,100000.0
...,...
2022-12-23,95169.0
2022-12-27,95169.0


In [11]:
len(srr.orders_processed)

NameError: name 'srr' is not defined

In [ ]:
len(srr.orders_unprocessed)